In [96]:
import freud
import gsd
import gsd.pygsd
import gsd.hoomd
import signac
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cycler
from scipy.stats import linregress
from matplotlib import cm
from pylab import *
import matplotlib.lines as mlines

In [97]:
def atom_type_pos(snap, atom_type):
    if not isinstance(atom_type, list):
        atom_type = [atom_type]
    positions = []
    for atom in atom_type:
        indices = np.where(snap.particles.typeid == snap.particles.types.index(atom))
        positions.append(snap.particles.position[indices])
    return np.concatenate(positions)

def msd_from_gsd(gsdfile, start=-10, stop=-1, atom_type='ss', msd_mode = "window"):
    f = gsd.pygsd.GSDFile(open(gsdfile, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    positions = []
    for frame in trajectory[start:stop]:
        if atom_type == 'all':
            atom_positions = frame.particles.position[:]
        else:
            atom_positions = atom_type_pos(frame, atom_type)
        positions.append(atom_positions)
    msd = freud.msd.MSD(box=trajectory[-1].configuration.box, mode=msd_mode)
    msd.compute(positions)
    f.close()
    return(msd.msd) # This is retruning an array of numbers (the y-axis on the msd plot)

In [98]:
project = signac.get_project("ptb7-project")

In [101]:
state_dict = {'size': 'small', 'process': 'quench', "density": 0.8, "molecule": "PTB7_5mer_smiles"}

job_list = project.find_jobs(state_dict)
for job in job_list:
    msd = msd_from_gsd(job.fn('trajectory.gsd'))
    y = msd[-5:]
    x = range(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    job.doc['msd_slope'] = slope
    job.doc['msd_slope_r2'] = r_value
    #print("slope of", job, "is:")
    #print(slope)
    
print(job.doc)

{'steps': 30000000.0, 'avogadro': 6.022140857e+23, 'boltzmann': 1.38064852e-23, 'kj_to_j': 1000.0, 'amu_to_kg': 1.6605e-27, 'nm_to_m': 1e-09, 'mass': 32.06, 'mass_units': 'amu', 'energy': 1.046, 'energy_units': 'kj/mol', 'distance': 0.35635948725613575, 'distance_units': 'nm', 'T_SI': 226.0, 'T_unit': 'K', 'real_timestep': 1.973, 'time_unit': 'fs', 'msd_slope': 16.540258603445764, 'msd_slope_r2': 0.8559580499476385}


In [1]:
project = signac.get_project("ptb7-project")



state_dict = {'size': 'small', 'process': 'quench', 
              "density": 0.8}


job_list = project.find_jobs(state_dict)
   
for idx,job in enumerate(job_list):
    msd = msd_from_gsd(job.fn('trajectory.gsd'))
    slope = job.doc['msd_slope']
    x2=job.sp["kT_reduced"]
    y2=slope
    mark = "x"
    #print(job.sp)
    if job.sp["molecule"] == 'PTB7_5mer_smiles':
        mark = "x"
    if job.sp["molecule"] == 'PTB7_10mer_smiles':
        mark = "v"
    if job.sp["molecule"] == 'PTB7_15mer_smiles':
        mark = "o"
    if x2 == 0.8: 
        col = ['#00004c']
    if x2 == 0.9: 
        col = ['#000079']
    if x2 == 1.0: 
        col = ['#0000a6']
    if x2 == 1.1: 
        col = ['#0000d2']
    if x2 == 1.2: 
        col = ['#0000ff']
    if x2 == 1.3: 
        col = ['#4040ff']
    if x2 == 1.4: 
        col = ['#8080ff']
    if x2 == 1.5: 
        col = ['#bfbfff']
    if x2 == 1.6: 
        col = ['#dbd9d9']
    if x2 == 1.7: 
        col = ['#ffbfbf']
    if x2 == 1.8: 
        col = ['#ff8080']
    if x2 == 1.9: 
        col = ['#ff4040']
    if x2 == 2.0: 
        col = ['#ff0000']
    if x2 == 2.1: 
        col = ['#df0000']
    if x2 == 2.2: 
        col = ['#bf0000']
    if x2 == 2.3: 
        col = ['#9f0000']
    if x2 == 2.4: 
        col = ['#800000']
    plt.ylim(0, 35.0)
    plt.scatter(x2,y2, marker=mark,color = col)#, marker=mpl.markers)
plt.minorticks_on()
plt.tick_params(axis='both', which='both', direction='in')
plt.title('Slope vs. Temp of 0.8 density')
plt.xlabel('Temperature (kT)')
plt.ylabel('Slope')

mer_5 = mlines.Line2D([], [], color='grey', marker='x', linestyle='None',
                      label='5mer')
mer_10 = mlines.Line2D([], [], color='grey', marker='v', linestyle='None',
                       label='10mer')
mer_15 = mlines.Line2D([], [], color='grey', marker='o', linestyle='None',
                       label='15mer')

plt.legend(handles=[mer_5, mer_10, mer_15], loc="upper left")

plt.show()


NameError: name 'signac' is not defined

In [25]:
#print(job.sp)

In [8]:
#counting # of jobs for scatter plot 
#for index,job in enumerate(job_list):    
#    x2=job.sp["kT_reduced"]
#    y2=slope
#    print(index, x2, y2)


#to get hex from colormap
#from pylab import *
#cmap = cm.get_cmap('seismic', 17)    # PiYG
#for i in range(cmap.N):
#    rgb = cmap(i)[:3] # will return rgba, we take only first 3 so we get rgb
#    print(matplotlib.colors.rgb2hex(rgb))